<h1 align="center">Machine learning-based prediction of early recurrence in glioblastoma patients: a glance towards precision medicine <br><br>[Logistic Regression]</h1>

## [1] Library

In [ ]:
# OS library
import os
import sys
import argparse

# Analysis
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Sklearn
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, recall_score, classification_report, accuracy_score, auc, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

import scikitplot as skplt
from imblearn.over_sampling import RandomOverSampler, SMOTENC, SMOTE

## [2] Data Preprocessing

<h4>[-] Load the database</h4>

In [ ]:
file = os.path.join(sys.path[0], "db.xlsx")
db = pd.read_excel(file)

print("N° of patients: {}".format(len(db)))
print("N° of columns: {}".format(db.shape[1]))
db.head()

<h4>[-] Drop unwanted columns + create <i>'results'</i> column</h4>

In [ ]:
df = db.drop(['Name_Surname', '...'], axis = 'columns')

print("Effective features to consider: {} ".format(len(df.columns)-1))
print("Creating 'result' column...")

# 0 = No relapse
df.loc[df['PFS'] > 6, 'result'] = 0

# 1 = Early relapse (within 6 months)
df.loc[df['PFS'] <= 6, 'result'] = 1

df.head()

<h4>[-] Label encoding of the categorical variables </h4>

In [ ]:
df['sex'] =df['sex'].astype('category')
df['ceus'] =df['ceus'].astype('category')
df['ala'] =df['ala'].astype('category')

#df['Ki67'] =df['Ki67'].astype(int)
df['MGMT'] =df['MGMT'].astype('category')
df['IDH1'] =df['IDH1'].astype('category')

df['side'] =df['side'].astype('category')
df['ependima'] =df['ependima'].astype('category')
df['cc'] =df['cc'].astype('category')
df['necrotico_cistico'] =df['necrotico_cistico'].astype('category')
df['shift'] =df['shift'].astype('category')

## VARIABLE TO ONE-HOT-ENCODE
df['localization'] =df['localization'].astype(int)
df['clinica_esordio'] =df['clinica_esordio'].astype(int)
df['immediate_p_o'] =df['immediate_p_o'].astype(int)
df['onco_Protocol'] =df['onco_Protocol'].astype(int)

df['result'] =df['result'].astype(int)

dummy_v = ['localization', 'clinica_esordio', 'onco_Protocol', 'immediate_p_o']

df = pd.get_dummies(df, columns = dummy_v, prefix = dummy_v)

## [3] Prediction Models

<h4> [-] Training and testing set splitting</h4>

In [ ]:
target = df['result']
inputs = df.drop(['result', 'PFS'], axis = 'columns')

Select columns (variable) at a univariate analysis ad a p-value lower than 0.05

In [ ]:
cols = ['age', 'EOR', 
        'onco_Protocol_0','onco_Protocol_1', 'onco_Protocol_2', 
        'onco_Protocol_3', 'onco_Protocol_5', 'MGMT', 
        'IDH1', 'volume_PreT2', 'residual_tumor', 
        'KPS_preop', 'KPS_postop']


x_train, x_test, y_train, y_test = train_test_split(inputs[cols],target,test_size=0.20, random_state=42)

<h4> [-] SMOTE-NC</h4>

In [ ]:
os = SMOTENC(categorical_features=[2,3,4,5,6,7,8], k_neighbors=4, random_state= 42)
smote_x,smote_y= os.fit_sample(x_train, y_train)

<h4> [-] Grid Search Hyperparameter tuning</h4>

In [ ]:
random_grid = [{'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]

# First create the base model to tune
lg = LogisticRegression(random_state=42)

# Random search of parameters, using 5 fold cross validation, different combinations, and use all available cores
lg_random = GridSearchCV(estimator = lg, param_grid=random_grid,
                               cv = 5)
# Fit the random search model
lg_random.fit(x_train, y_train)
lg_random.best_params_

<h4> [-] Logistic Regression</h4>

In [ ]:
log_pfs = LogisticRegression(random_state=42, penalty='l2', C=10)
log_pfs.fit(smote_x, smote_y)

score_log = log_pfs.score(x_test, y_test)
print("### TESTING ###")
print("Logistic Regression's accuracy: ", round(score_log*100,2), "% \n")

y_pred = log_pfs.predict(x_test)
y_proba = log_pfs.predict_proba(x_test)
cm_log = confusion_matrix(y_test, y_pred)
print(cm_log, "\n")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)


print('1. The F-1 Score of the model {} \n '.format(round(f1_score(y_test, y_pred, average = 'macro'), 2)))
print('2. The Recall Score of the model {} \n '.format(round(recall_score(y_test, y_pred, average = 'macro'), 2)))
print('3. Classification report \n {}'.format(classification_report(y_test, y_pred)))
print('3. AUC: \n {} \n'.format(roc_auc))

tn, fp, fn, tp = cm_log.ravel()

# Sensitivity, hit rate, Recall, or true positive rate
tpr = tp/(tp+fn)
print("Sensitivity (TPR): {}".format(tpr))

# Specificity or true negative rate
tnr = tn/(tn+fp)
print("Specificity (TNR): {}".format(tnr))

# Precision or positive predictive value
ppv = tp/(tp+fp)
print("Precision (PPV): {}".format(ppv))

# Negative predictive value
npv = tn/(tn+fn)
print("Negative Predictive Value (NPV): {}".format(npv))

# False positive rate
fpr = fp / (fp + tn)
print("False Positive Rate (FPV): {}".format(fpr))